In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
train_dir = 'C:/Users/harish/Desktop/XPressiify/backend/models/archive/archive/train'
test_dir = 'C:/Users/harish/Desktop/XPressiify/backend/models/archive/archive/test'

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, zoom_range=0.2, shear_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)


Found 36289 images belonging to 7 classes.
Found 7935 images belonging to 7 classes.


In [3]:
# Create the model
model = Sequential()

# First convolutional layer
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(48, 48, 1)))

# Second convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))

# Third convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))

# MaxPooling and Dropout
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))

# Fourth convolutional layer
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))

# MaxPooling and Dropout
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))

# Fifth convolutional layer
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))

# MaxPooling and Dropout
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))

# Flatten the layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer for classification (7 emotion classes)
model.add(Dense(7, activation='softmax'))
learning_rate=0.0001
optimizer=Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\harish\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 408s 360ms/step - accuracy: 0.5854 - loss: 1.1026 - val_accuracy: 0.5922 - val_loss: 1.1445
Epoch 2/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 499s 439ms/step - accuracy: 0.5917 - loss: 1.0827 - val_accuracy: 0.5941 - val_loss: 1.1321
Epoch 3/10
 429/1135 ━━━━━━━━━━━━━━━━━━━━ 12:20 1s/step - accuracy: 0.5958 - loss: 1.0630

In [11]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")
print(f"Test Loss: {test_loss}")

248/248 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - accuracy: 0.6260 - loss: 1.1746
Test Accuracy: 0.6293635964393616
Test Loss: 1.153333067893982


In [8]:
# Save the fine-tuned model
model.save('emo.h5')


In [12]:
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Step 1: Load the pre-trained model
model = load_model('emo.h5')

# Step 2: (Optional) Freeze the earlier layers to prevent them from being re-trained
for layer in model.layers[:-3]:  # Adjust the number depending on which layers you want to freeze
    layer.trainable = False

# Step 3: Compile the model with a lower learning rate for fine-tuning
optimizer = Adam(learning_rate=0.00001)  # Fine-tuning often works better with a lower learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Set callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Step 5: Continue training (fine-tuning)
history = model.fit(
    train_generator,
    epochs=20,  # You can adjust this
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Step 6: Evaluate the fine-tuned model on the test set
#test_loss, test_accuracy = model.evaluate(test_generator)
#print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/20
 209/1135 ━━━━━━━━━━━━━━━━━━━━ 3:16 213ms/step - accuracy: 0.6970 - loss: 0.8051

KeyboardInterrupt: 

In [7]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


NameError: name 'history' is not defined